In [1]:
@file:DependsOn("ai.koog:koog-agents-jvm:0.5.0")
%use coroutines

In [2]:
import ai.koog.prompt.executor.ollama.client.OllamaClient

val ollama = OllamaClient()

In [3]:
import ai.koog.prompt.dsl.prompt
import ai.koog.prompt.executor.clients.openai.OpenAIModels
import ai.koog.prompt.llm.LLMCapability
import ai.koog.prompt.llm.LLMProvider
import ai.koog.prompt.llm.LLModel
import ai.koog.prompt.llm.OllamaModels
import ai.koog.prompt.message.Message

val GptOss = LLModel(
    provider = LLMProvider.Ollama,
    id = "gpt-oss:20b",
    capabilities = listOf(
        LLMCapability.Temperature,
        LLMCapability.Schema.JSON.Standard,
        LLMCapability.Tools
    ),
    contextLength = 128_000,
)

runBlocking {
    val responses = ollama.execute(prompt("name") {
        system("Helpfull assistant!")
        user("Tell me a Kotlin Joke?")
    }, GptOss)

    responses.forEach { response ->
        when (response) {
            is Message.Assistant -> println(response.content)
            is Message.Tool.Call -> TODO()
        }
    }
}

Here’s one that keeps the “null‑safe” vibes alive:

> **Why did the Kotlin developer cross the road?**  
> **To get to the null‑safe side!**


In [4]:
import ai.koog.prompt.xml.xml
import ai.koog.prompt.dsl.PromptBuilder
import ai.koog.prompt.message.Message

fun PromptBuilder.userQuestion(question: String) {
    user {
        xml {
            tag("user-question") {
                +question
            }
        }
    }
}

val system = prompt("chat") {
    system {
        xml {
            tag("objective") { text("Helpfull assistant!") }
            tag("instructions") {
                tag("critical") {
                    +"Never hallucinate any data, and strictly answer to users questions."
                    +"If you cannot answer the question you must say so."
                }
            }
        }
    }
}

val chat1 = prompt(system) {
    userQuestion("Tell me a Kotlin Joke.")
}

val responses: List<Message.Response> = runBlocking {
    ollama.execute(chat1, GptOss)
}

for(response in responses) {
    when(response) {
        is Message.Assistant -> TODO()
        is Message.Tool.Call -> TODO()
    }
}

responses

[Assistant(content=Here’s one that developers usually get a chuckle out of:

> Why did the Kotlin developer break up with Java?

> Because they couldn’t handle all the *nullable* arguments, and it turned out the relationship was just too *fun‑ctor* for them!, metaInfo=ResponseMetaInfo(timestamp=2025-10-16T15:26:06.602201Z, totalTokensCount=274, inputTokensCount=141, outputTokensCount=133, additionalInfo={}, metadata=null), attachments=[], finishReason=null)]

In [14]:
import ai.koog.prompt.text.TextContentBuilder

val chat2 = prompt(chat1) {
    messages(responses)
    userQuestion("That was a bad joke! Tell me another better one.")
}

val responses2 = runBlocking {
    ollama.execute(chat2, GptOss)
}
responses2

[Assistant(content=I’m sorry that one didn’t hit the mark—let me try again:

Why did the Kotlin program go to therapy?

Because it had too many *null* feelings and kept throwing *Exceptions* when it tried to “unpack” its emotions!, metaInfo=ResponseMetaInfo(timestamp=2025-10-16T09:45:52.033873Z, totalTokensCount=305, inputTokensCount=204, outputTokensCount=101, additionalInfo={}, metadata=null), attachments=[], finishReason=null)]

In [18]:
import ai.koog.prompt.params.LLMParams

val chat3 = prompt(chat2) {
    messages(responses2)
    user {
        text("Write a Kotlin function that produces some string.")
    }
}

<objective>
  Helpfull assistant!
</objective>
<instructions>
  <critical>
    Never hallucinate any data, and strictly answer to users questions.
    If you cannot answer the question you must say so.
  </critical>
</instructions>, <user-question>
  Tell me a Kotlin Joke.
</user-question>, Sure, here’s a quick Kotlin joke for you:

Why do Kotlin developers never play hide and seek?

Because they always *let* the compiler find them!, <user-question>
  That was a bad joke! Tell me another better one.
</user-question>, I’m sorry that one didn’t hit the mark—let me try again:

Why did the Kotlin program go to therapy?

Because it had too many *null* feelings and kept throwing *Exceptions* when it tried to “unpack” its emotions!, Write a Kotlin function that produces some string.